# How to Create Pivot Tables With pandas

[Fuente](https://realpython.com/how-to-pandas-pivot-table/)

[Data](https://www.kaggle.com/datasets/satyamanidharv/sales-data-presentation-dashboards?resource=download)

## Tabla Markdown para datos de pedidos

| Nombre de columna | Tipo de datos (PyArrow) | Descripción |
|---|---|---|
| order_number | int64 | Número de pedido (único) |
| employee_id | int64 | Identificador del empleado (único) |
| employee_name | string | Nombre completo del empleado |
| job_title | string | Cargo del empleado |
| sales_region | string | Región de ventas en la que trabaja el empleado |
| order_date | timestamp[ns] | Fecha en que se realizó el pedido |
| order_type | string | Tipo de pedido (Minorista o Mayorista) |
| customer_type | string | Tipo de cliente (Empresa o Individual) |
| customer_name | string | Nombre completo del cliente |
| customer_state | string | Estado de residencia del cliente |
| product_category | string | Categoría del producto (Productos de baño, Cesta de regalo, Aceite de oliva) |
| product_number | string | Identificador del producto (único) |
| product_name | string | Nombre del producto |
| quantity | int64 | Cantidad ordenada |
| unit_price | double | Precio de venta de un producto |
| sale_price | double | Precio total de venta (unit_price × quantity) |



 Here we use the pandas library to allow you to work with DataFrames and the newer PyArrow library. The PyArrow library provides pandas with its own optimized data types, which are faster and less memory-intensive than the traditional NumPy types pandas uses by default.

In [1]:
import pandas as pd

sales_data = pd.read_csv(
    "sales_data.csv",
    parse_dates=["order_date"],
    dayfirst=True,
    dtype_backend="pyarrow"
)

In [2]:
sales_data.head()

,order_number,employee_id,employee_name,job_title,sales_region,order_date,order_type,customer_type,customer_name,customer_state,product_category,product_number,produce_name,quantity,unit_price,sale_price
0,1102935,900019019,Alexandra Kundt,Senior Sales Associate,S Central East,2019-02-09,Retail,Individual,Skipton Fealty,Arkansas,Olive Oil,OO206,Chili Extra Virgin Olive Oil 2pk,3,45.0,135.0
1,1102976,900019019,Alexandra Kundt,Senior Sales Associate,S Central East,2019-02-15,Retail,Individual,Lanni D'Ambrogi,Missouri,Gift Basket,GB301,Scented Olive Oil Candle Gift Basket,1,19.5,19.5
2,1102988,900019019,Alexandra Kundt,Senior Sales Associate,S Central East,2019-02-16,Retail,Individual,Far Pow,Mississippi,Olive Oil,OO302,Chili Extra Virgin Olive Oil,4,26.0,104.0
3,1103012,900019019,Alexandra Kundt,Senior Sales Associate,S Central East,2019-02-19,Wholesale,Business,Swift Inc,Texas,Olive Oil,OO125,Garlic Extra Virgin Olive Oil 12pk,4,234.0,936.0
4,1103031,900019019,Alexandra Kundt,Senior Sales Associate,S Central East,2019-02-22,Retail,Individual,Carmine Priestnall,Texas,Olive Oil,OO128,Chili Extra Virgin Olive Oil 12pk,3,234.0,702.0


In [3]:
sales_data.dtypes

order_number         int64[pyarrow]
employee_id          int64[pyarrow]
employee_name       string[pyarrow]
job_title           string[pyarrow]
sales_region        string[pyarrow]
order_date           datetime64[ns]
order_type          string[pyarrow]
customer_type       string[pyarrow]
customer_name       string[pyarrow]
customer_state      string[pyarrow]
product_category    string[pyarrow]
product_number      string[pyarrow]
produce_name        string[pyarrow]
quantity             int64[pyarrow]
unit_price          double[pyarrow]
sale_price          double[pyarrow]
dtype: object

First of all, you used import pandas to make the library available within your code. To construct the DataFrame and read it into the sales_data variable, you used pandas’ read_csv() function. The first parameter refers to the file being read, while parse_dates highlights that the order_date column’s data is intended to be read as the datetime64[ns] type. But there’s an issue that will prevent this from happening.

In your source file, the order dates are in dd/mm/yyyy format, so to tell read_csv() that the first part of each date represents a day, you also set the dayfirst parameter to True. This allows read_csv() to now read the order dates as datetime64[ns] types.

With order dates successfully read as datetime64[ns] types, the .convert_dtypes() method can then successfully convert them to a timestamp[ns][pyarrow] data type, and not the more general string[pyarrow] type it would have otherwise done. Although this may seem a bit circuitous, your efforts will allow you to analyze data by date should you need to do this.

**Note:** If you’re experienced in data analysis, you’re no doubt aware of the need for data cleansing. This is still important as you work with pivot tables, but it’s equally important to make sure your input data is also tidy.

**Tidy data** is organized as follows:

- Each **row** should contain a single record or observation.
- Each **column** should contain a single observable or variable.
- Each **cell** should contain an atomic value.

If you tidy your data in this way, as part of your data cleansing, you’ll also be able to analyze it better. For example, rather than store address details in a single address field, it’s usually better to split it down into *house_number*, *street_name*, *city*, and *country* component fields. This allows you to analyze it by individual streets, cities, or countries more easily.

In addition, you’ll also be able to use the data from individual columns more readily in calculations. For example, if you had columns *room_length* and *room_width*, they can be multiplied together to give you room area information. If both values are stored together in a single column in a format such as "10 x 5", the calculation becomes more awkward.

The data within the *sales_data.csv* file is already in a suitably clean and tidy format for you to use in this tutorial. However, not all raw data you acquire will be.

## How to Create Your First Pivot Table With pandas

Now that your learning journey is underway, it’s time to progress toward your first learning milestone and complete the following task:

**Calculate the total sales for each type of order for each region.**

Before you create a pivot table, you should first consider how you want the data to be displayed. One approach would be to have a separate column for each product category and a separate row for each region. The total sales for each product category for each region could then be placed in the intersection of each row and column.

 You need to translate your plan into the parameters required by the DataFrame.pivot_table() method. In this case, the parameters you need are **values**, **index**, **columns**, and **aggfunc**. It’s important to understand what these are for because they’re the core parameters used in most pivot tables.

 Your pivot table will perform its calculations based on the sale price figures. These become the values passed to the **values** parameter, giving your pivot table the figures it needs to work with. If you don’t specify this parameter, the pivot table will use all numeric columns by default.

 The **index** parameter is where you specify how the data is to be grouped. Remember, you need to find the total sales for each type of order for each region. You’ve decided to produce a row of aggregated data based on the *sales_region* field, so this will be passed as the index parameter.

 You also want a separate column for each order type. This will be assigned to the **columns** parameter.

 Finally, you also need to tell your pivot table that you want to calculate the total sales for each aggregation. To create a simple pivot table, you set its **aggfunc** parameter to, in this case, the sum() function.

 Now that you’ve fully thought through your plan, you can go ahead and code your pivot table:

In [14]:
pd.set_option("display.float_format", "${:,.2f}".format)

sales_data.pivot_table(
    values="sale_price", index="sales_region", columns="order_type",
    aggfunc="sum",
)

order_type,Retail,Wholesale
sales_region,,
Central East,"$102,613.51","$149,137.89"
N Central East,"$117,451.69","$152,446.42"
N Central West,"$10,006.42","$1,731.50"
Northeast,"$84,078.95","$127,423.36"
Northwest,"$34,565.62","$33,240.12"
S Central East,"$130,742.32","$208,945.73"
S Central West,"$54,681.80","$51,051.03"
Southeast,"$96,310.12","$127,554.60"
Southwest,"$104,743.52","$121,977.20"


As you can see, .pivot_table() used the parameters you planned earlier to produce your pivot table.

To format the numbers as currency, you used Python’s string format mini-language. This language is used in several places in Python, most commonly in f-strings. If you’re already familiar with it in that context, you already have the heads-up on how to use it here.

By passing "display.float_format" and "\${:,.2f}".format to pandas" set_option() function, you define the format for floating-point numbers from this point forward. They will be rounded to 2 decimal places, use a comma (,) as their thousands separator, and be prefixed with a ($) currency symbol.

Unless you want to keep this format for your future floating-point numbers, you’ll need to reset the formatting to its default by using pd.reset_option("display.float_format") after .pivot_table() has been called.

While you’re basically happy with your result, you feel that something is missing: totals. You hadn’t initially thought about including totals columns, but now realize this would be useful. To fix this, you consult the documentation and come up with the following solution:

In [15]:
pd.set_option("display.float_format", "${:,.2f}".format)

sales_data.pivot_table(
    values="sale_price", index="sales_region", columns="order_type",
    aggfunc="sum", margins=True, margins_name="Totals"
)

order_type,Retail,Wholesale,Totals
sales_region,,,
Central East,"$102,613.51","$149,137.89","$251,751.40"
N Central East,"$117,451.69","$152,446.42","$269,898.11"
N Central West,"$10,006.42","$1,731.50","$11,737.92"
Northeast,"$84,078.95","$127,423.36","$211,502.31"
Northwest,"$34,565.62","$33,240.12","$67,805.74"
S Central East,"$130,742.32","$208,945.73","$339,688.05"
S Central West,"$54,681.80","$51,051.03","$105,732.83"
Southeast,"$96,310.12","$127,554.60","$223,864.72"
Southwest,"$104,743.52","$121,977.20","$226,720.72"


This time, to add some additional finishing touches, you set margins=True and margins_name="Totals". The parameter margins=True added new columns onto the right and bottom of your pivot table. Each contains the totals of the rows and columns, respectively. The margins_name parameter inserted "Totals" labels instead of the default "All" labels that would appear otherwise.

**Note**: When you create a pivot table using .pivot_table(), what gets returned is a new DataFrame. This means that anything you usually do with your DataFrames you can also do with your pivot tables. For example, you can save them to files or use them in plots.